In [2]:
#export
#import torch
import torch.nn as nn
from fastai.vision.all import *
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
import time
from datetime import date
from vidaug import augmentors as va

import torchvision.transforms as T


from modules_th.video_block import * 
from modules_th.inflator import *
from modules_th.triplet_loss import *
from modules_th.unsupcon_module import *
from modules_th.cus_cbs import *

#Pretrained models
from modules_th.pretrained_r2p1d50 import *
from resnetmodels.mean import get_mean_std

In [12]:
#export
def read_data():
    path = '/mnt/data/eugeniomarinelli/UCF_Crimes/normal_videos.csv'
    return pd.read_csv(path, index_col=0)

In [13]:
df = read_data()
df = df.sample(frac=0.1)
# train_idxs,val_idxs = RandomSplitter()(df)
# df.val.iloc[train_idxs]= False
# df.val.iloc[val_idxs]= True
# df.to_csv('/mnt/data/eugeniomarinelli/UCF_Crimes/normal_videos.csv')


In [14]:
df.tail()

,vid_files,val,embs_path
1301,/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Training_Normal_Videos_Anomaly/Normal_Videos132_x264.mp4,False,NaN
288,/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Training_Normal_Videos_Anomaly/Normal_Videos862_x264.mp4,False,/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Training_Normal_Videos_Anomaly/Normal_Videos862_x264
1664,/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Training_Normal_Videos_Anomaly/Normal_Videos747_x264.mp4,False,NaN
1501,/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Training_Normal_Videos_Anomaly/Normal_Videos695_x264.mp4,False,NaN
117,/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Training_Normal_Videos_Anomaly/Normal_Videos638_x264.mp4,False,/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Training_Normal_Videos_Anomaly/Normal_Videos638_x264


In [23]:
# export
def sometimes(x):
    return va.Sometimes(0.5,x)
    
def wrapVideo(x):
    return Video(x)

def unsuplabel(x):
    return 1

In [36]:
#export
def get_dsets(df, l=20, size=512,skip=20,n_views=2):
    vid_paths = df.vid_files.values
    prefix = Path('/mnt/data/eugeniomarinelli/UCF_Crimes/Videos')
    vid_paths = [prefix/file for file in vid_paths]
    
    
    vid_pip = [createVideoForm(l=l,skip=skip, form='img'), 
               Resize(size, method=ResizeMethod.Pad),
               sometimes(va.HorizontalFlip()),
               va.GaussianBlur(1.),
#               sometimes(va.InvertColor()),
               va.RandomRotate(10),
               wrapVideo]
    
    pip = [*([vid_pip]*n_views),[unsuplabel, Categorize]]
    #splits
    splits = ColSplitter('val')(df)

    # Datasets and dataloaders
    dsets = Datasets(vid_paths, pip, splits=splits)
    return dsets, splits

In [37]:
ColSplitter('val')(df)

((#62) [0,1,2,4,5,6,8,9,10,12...], (#10) [3,7,11,19,22,34,52,65,67,71])

In [38]:
dsets,splits = get_dsets(df)
vid = dsets[0][0]

In [39]:
len(dsets[0])

3

In [40]:
# export
def get_dls(dsets,splits,df, bs= 4, normalize='kinetics'):
        
    mean, std = get_mean_std(1,normalize)
    dls  = dsets.dataloaders(bs=bs, 
                             shuffle_train=True,
                             after_item=ToTensor(),
                             after_batch=[IntToFloatTensor(), Normalize.from_stats(*imagenet_stats)])
    
    dls.valid.shuffle = True
    return dls
    

In [41]:
dls = get_dls(dsets,splits,df, normalize='kinetics')


In [42]:
#export
def setup_log(learn,name, append=True):
     # set up logs file
    # now = datetime.now()
    # time = now.strftimes("%d_%m")
    logs_file = '/mnt/data/eugeniomarinelli/UCF_experiments/training_results/logs_UCF_CRIMES_'+name+'.csv'
    Logs_csv =   CSVLogger(fname= logs_file, append=append)    
    learn.add_cb(Logs_csv)

In [43]:
#export
def get_learner(df,
                pretrained_model='r2p1d50_K', 
                l=40, size=224, bs = 4, skip=20, embs_size=256,n_views=2, 
                normalize = 'kinetics', logs=True):

    dsets,splits = get_dsets(df, l, size, skip, n_views)

    dls = get_dls(dsets,splits,df, normalize=normalize)
    
    if pretrained_model in inserted_models:
        model = inserted_models[pretrained_model]
    else: raise 'model not present in pretrained models'
   

    body = create_body(model, cut=-2) 
        

    Loss= ContrastiveLoss()
    head = inflate(create_head(4096, embs_size, lin_ftrs=[]))
    model = nn.Sequential(body,head)

    
        
    learn = Learner(dls, 
                model,
                splitter=splitter ,
                loss_func=Loss)

    learn.add_cb(ContrastiveCallback(n_views))
    
    time = date.today().strftime("_%d-%m")
    if logs:    
        setup_log(learn, str(pretrained_model)+'unsupervised'+'5fr_30unf'+time, append=True)
   
    return learn
 

In [44]:
class UnsupContrastiveCallback(Callback):
    def __init__(self, nviews):
        super().__init__()
        self.nviews = nviews
    def after_pred(self): 
        stack_views(self)
#        self.learn.yb = torch.tensor([None for _ in range(4)])
    def before_batch(self): cat_views(self)

In [45]:
learn = get_learner(df, 'r2p1d50_K', l=2, size=64, bs= 4, skip=20, embs_size=256, n_views=2, logs=False)
#learn.add_cb(CSVLogger(fname='data/eugeniomarinelli/UCF_experiments/trained_models_cnn/learners/LOGGER_PROVA' , append=False)  )

loading pretrained model /mnt/data/eugeniomarinelli/pretrained/r2p1d50_K_200ep.pth


In [46]:
##learn.recorder = Recorder(valid_metrics=False)
learn.loss_func

ContrastiveLoss()

In [47]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time
0,2.443388,0.831255,00:07


enter in loss
enter in loss
enter in loss
enter in loss
enter in loss
enter in loss
enter in loss
enter in loss
enter in loss
enter in loss
enter in loss
enter in loss
enter in loss
enter in loss
enter in loss
enter in loss
enter in loss
enter in loss


In [29]:
learn.loss

In [ ]:
#export
def save_learner(learn, name):
    prefix = '/mnt/data/eugeniomarinelli/'
    try:
        learn.export(prefix+'UCF_experiments/trained_models_cnn/learners/learner_ucf101_'+name)
    except: print("learner export didn't work") 
    try:
        torch.save(learn.model,prefix+'UCF_experiments/trained_models_cnn/models/model_ucf101_'+name+'.pth')
    except: torch.save(learn.model.state_dict(),prefix+'UCF_experiments/trained_models_cnn/models/state_dict_ucf101_'+name)
    


In [ ]:
learn.export('/mnt/data/eugeniomarinelli/UCF_experiments/trained_models_cnn/learners/LEARNER_CANCELLARE')
torch.save(learn.model, '/mnt/data/eugeniomarinelli/UCF_experiments/trained_models_cnn/models/MODEL_CANCELLARE')

In [ ]:
b = torch.load('/mnt/data/eugeniomarinelli/UCF_experiments/trained_models_cnn/learners/learner_ucf101_XXXXXX.pth')

In [ ]:
saved_model_SCL = torch.load('/mnt/data/eugeniomarinelli/UCF_experiments/trained_models_cnn/models/r2p1d50_ucf101_SCL_tuned_15fr.pth')

In [ ]:

model = nn.Sequential(saved_model_SCL,nn.Sequential(nn.ReLU(inplace=True),LinBnDrop(256, 101, p=0.5)))


In [ ]:
len(splitter(model))

2

In [ ]:
saved_model.head

Sequential(
  (0): AdaptiveConcatPool3d(
    (ap): AdaptiveAvgPool3d(output_size=1)
    (mp): AdaptiveMaxPool3d(output_size=1)
  )
  (1): Flatten(full=False)
  (2): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=4096, out_features=256, bias=False)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=256, out_features=101, bias=False)
)

In [15]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_losses_tmp.ipynb.
Converted 02_inflator.ipynb.
Converted 03_video_block.ipynb.
Converted 05_charades.ipynb.
Converted 07_read_video_function.ipynb.
Converted 08_transformer.ipynb.
Converted 09_lr_schedule.ipynb.
Converted 10_experiments_on_images.ipynb.
Converted 11_create_embeddings.ipynb.
Converted 12_load_r2p1d_model.ipynb.
Converted CALTECH101_MixedLoss.ipynb.
Converted CALTECH101_singleloss.ipynb.
Converted Embeddings_CRIMES.ipynb.
Converted Embeddings_UCF101.ipynb.
Converted UCF101.ipynb.
Converted UCF101_unsupervised.ipynb.
Converted UCF_Crimes_CEL.ipynb.
Converted UCF_Crimes_SupCon.ipynb.
Converted index.ipynb.


In [ ]:
#default_exp ucfcrimes_unsup

Path('/mnt/data/eugeniomarinelli/videodata/UCF-101')

RuntimeError: [enforce fail at inline_container.cc:222] . file not found: archive/data.pkl